In [1]:
#https://chromedriver.chromium.org/downloads - download based on your version of chrome - chrome://settings/help
#you will need to install lxml - pip install lxml
# https://www.psycopg.org/docs/ - pip install psycopg
import pyotp
import datetime
import time as ti
import requests
from requests.exceptions import ConnectionError
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
from selenium import webdriver
from selenium.common.exceptions import NoAlertPresentException
stock_code = 'NVDA'
driver = webdriver.Chrome()
url = 'https://finance.yahoo.com/quote/' + stock_code + '?p=' + stock_code + '&.tsrc=fin-srch'
r = requests.get(url) 
    
while True:
    stock_code = 'NVDA'
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36",
    "Cache-Control": "no-cache, max-age=0",  # disable caching
}
    
    A = 'My(6px) Pos(r) smartphone_Mt(6px) W(100%)'
    B = 'D(ib) Mend(20px)'
    C = 'Fw(b) Fz(36px) Mb(-4px) D(ib)'
    D = 'W(100%)'
    E = 'Fz(xs) Mb(4px)'
    F = 'W(100%) M(0) Bdcl(c)'
    G = ''
    H = ''
    I = ''
    J = ''
    K = ''
    L = ''
    M = ''
    N = ''
    O = ''
    P = ''
    Q = ''
    
    #driver.get()
    #r = requests.get(url, timeout = 0.001)
    
    def web_content_div(web_content, class_path):
        
        web_content_div = web_content.find_all('div', {'class': class_path})
        try:
            spans = web_content_div[0].find_all('span')
            text = [span.get_text() for span in spans]
        except IndexError:
            text = []
            
        return text
    def price(stock_code):
        try:
            r = requests.get(url)
            web_content = BeautifulSoup(requests.get(url, headers=headers).text, 'lxml')
            Price = web_content_div(web_content, B)
            PrevCl = web_content_div(web_content, G)
            DRange = web_content_div(web_content, H)
            SectionA = web_content_div(web_content, I)
        except ConnectionError:
            print('Something went wrong :(')
        return Price #, SectionA
    print(price(stock_code))
    #Result = pd.DataFrame(web_content_div(web_content, A))
    #print(Result)
    #driver.execute_script("document.body.innerHTML += Result")
    #driver.refresh()
    ti.sleep(5)
    

    #if text != []:
        #RegMarkPrice, DV1, PCT1, PostMarkPrice, DV2, PCT2 = texts[0], texts[1], texts[2], texts[6], texts[7], texts[8]
        
    #return RegMarkPrice, DV1 +' '+ PCT1, PostMarkPrice, DV2 +' '+ PCT2 #, volume, latest_pattern, one_year_target

['+0.77', '(+0.46%)', 'As of  12:13PM EDT. Market open.']
['+0.85', '(+0.51%)', 'As of  12:13PM EDT. Market open.']
['+0.66', '(+0.40%)', 'As of  12:13PM EDT. Market open.']
['+0.60', '(+0.36%)', 'As of  12:13PM EDT. Market open.']


KeyboardInterrupt: 

In [18]:
(['-4.30', '(-2.51%)', 'At close:  04:00PM EDT'], ['Sign in', '', 'Mail', 'U.S. markets closed', '+0.57', '(+0.01%)', '+8.80', '(+0.03%)', '-33.88', '(-0.30%)', '-2.95', '(-0.17%)', '+0.07', '(+0.06%)', 'NasdaqGS - NasdaqGS Real Time Price. Currency in USD', 'Add to watchlist', 'Add to watchlist', 'Visitors trend', 'Visitors trend', '2W', '2W', '10W', '10W', '9M', '9M', '-4.30', '(-2.51%)', 'At close:  04:00PM EDT', '0.00 (0.00%)', '0.00', '(0.00%)', 'After hours: 07:59PM EDT', 'After hours:', 'Summary', 'Company Outlook', 'Chart', 'Conversations', 'Statistics', 'Historical Data', 'Profile', 'Financials', 'Analysis', 'Options', 'Holders', 'Sustainability', 'Full screen', 'Full screen', 'Trade prices are not sourced from all markets', 'Gain actionable insight from technical analysis on financial instruments, to help optimize your trading strategiesChart Events', 'Gain actionable insight from technical analysis on financial instruments, to help optimize your trading strategies', 'Bullish', 'Bullish', 'pattern detected', 'Short Term', 'Mid Term', 'Long Term', 'Previous Close', 'Open', 'Bid', 'Ask', "Day's Range", '52 Week Range', 'Volume', 'Avg. Volume', 'Market Cap', 'Beta (5Y Monthly)', 'PE Ratio (TTM)', 'EPS (TTM)', 'Earnings Date', 'May 25, 2022', 'Forward Dividend & Yield', 'Ex-Dividend Date', 'Mar 02, 2022', '1y Target Est', 'Fair Value is the appropriate price for the shares of a company, based on its earnings and growth rate also interpreted as when P/E Ratio = Growth Rate. Estimated return represents the projected annual return you might expect after purchasing shares in the company and holding them over the default time horizon of 5 years, based on the EPS growth rate that we have projected.Fair Value', 'Fair Value is the appropriate price for the shares of a company, based on its earnings and growth rate also interpreted as when P/E Ratio = Growth Rate. Estimated return represents the projected annual return you might expect after purchasing shares in the company and holding them over the default time horizon of 5 years, based on the EPS growth rate that we have projected.', 'Subscribe to Yahoo Finance Plus to view Fair Value for NVDALearn moreView details', 'Subscribe to Yahoo Finance Plus to view Fair Value for NVDALearn more', 'Learn more', 'View details', 'View details', 'Research that delivers an independent perspective, consistent methodology and actionable insightRelated Research', 'Research that delivers an independent perspective, consistent methodology and actionable insight', "NVIDIA CorporationAnalyst Report: NVIDIA CorporationNvidia is the top designer of discrete graphics processing units that enhance the experience on computing platforms. The firm's chips are used in a variety of end markets, including high-end PCs for gaming, data centers, and automotive infotainment systems. In recent years, the firm has broadened its focus from traditional PC graphics applications such as gaming to more complex and favorable opportunities, including artificial intelligence and autonomous driving, which leverage the high-performance capabilities of the firm's graphics processing units. RatingFair ValueEconomic Moat7 days agoMorningstarAnalyst Report: NVIDIA CorporationNvidia is the top designer of discrete graphics processing units that enhance the experience on computing platforms. The firm's chips are used in a variety of end markets, including high-end PCs for gaming, data centers, and automotive infotainment systems. In recent years, the firm has broadened its focus from traditional PC graphics applications such as gaming to more complex and favorable opportunities, including artificial intelligence and autonomous driving, which leverage the high-performance capabilities of the firm's graphics processing units. 7 days ago•Morningstar", "NVIDIA CorporationAnalyst Report: NVIDIA CorporationNvidia is the top designer of discrete graphics processing units that enhance the experience on computing platforms. The firm's chips are used in a variety of end markets, including high-end PCs for gaming, data centers, and automotive infotainment systems. In recent years, the firm has broadened its focus from traditional PC graphics applications such as gaming to more complex and favorable opportunities, including artificial intelligence and autonomous driving, which leverage the high-performance capabilities of the firm's graphics processing units. RatingFair ValueEconomic Moat7 days agoMorningstar", 'Rating', '', 'Fair Value', '', 'Economic Moat', '', '7 days ago', 'Morningstar', '7 days ago', 'Morningstar', 'View more', 'All', 'News', 'Press Releases', 'Research Reports', 'SEC Filings', 'Hedge Fund Legend Predicts the Next 1,000% Winner', '', '', '', '', '', '', '', '', '', '', '', '© 2022 Yahoo. All rights reserved.', '© 2022 Yahoo. All rights reserved.', 'Data Disclaimer', 'Help', 'Suggestions', 'Privacy Dashboard', 'Privacy (Updated)', 'About Our Ads', 'Terms (Updated)', 'Sitemap'], [])

(['-4.30', '(-2.51%)', 'At close:  04:00PM EDT'],
 ['Sign in',
  '',
  'Mail',
  'U.S. markets closed',
  '+0.57',
  '(+0.01%)',
  '+8.80',
  '(+0.03%)',
  '-33.88',
  '(-0.30%)',
  '-2.95',
  '(-0.17%)',
  '+0.07',
  '(+0.06%)',
  'NasdaqGS - NasdaqGS Real Time Price. Currency in USD',
  'Add to watchlist',
  'Add to watchlist',
  'Visitors trend',
  'Visitors trend',
  '2W',
  '2W',
  '10W',
  '10W',
  '9M',
  '9M',
  '-4.30',
  '(-2.51%)',
  'At close:  04:00PM EDT',
  '0.00 (0.00%)',
  '0.00',
  '(0.00%)',
  'After hours: 07:59PM EDT',
  'After hours:',
  'Summary',
  'Company Outlook',
  'Chart',
  'Conversations',
  'Statistics',
  'Historical Data',
  'Profile',
  'Financials',
  'Analysis',
  'Options',
  'Holders',
  'Sustainability',
  'Full screen',
  'Full screen',
  'Trade prices are not sourced from all markets',
  'Gain actionable insight from technical analysis on financial instruments, to help optimize your trading strategiesChart Events',
  'Gain actionable insight fr